<a href="https://colab.research.google.com/github/Meguazy/HACK_4DS/blob/main/notebooks_stas/lgb_no_processing_day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Meguazy/HACK_4DS.git

Cloning into 'HACK_4DS'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 77 (delta 28), reused 1 (delta 0), pack-reused 0
Receiving objects: 100% (77/77), 11.56 MiB | 3.95 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [2]:
%cd /content/HACK_4DS

/content/HACK_4DS


# Initial

In [11]:
!pip install xgboost imbalanced-learn

In [20]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [5]:
df_test = pd.read_csv("data/test_set.csv", sep=";", quotechar="\"", decimal=",")

In [6]:
df_train = pd.read_csv("data/train_set.csv", sep=";", quotechar="\"", decimal=",")

# Preprocessing

In [7]:
external_score_ver03_dictionary = {
    "MISSING":None,
    "A":14,
    "B":13,
    "C":12,
    "D":11,
    "E":10,
    "F":9,
    "G":8,
    "H":7,
    "I":6,
    "L":5,
    "M":4,
    "N":3,
    "O":2,
    "P":1
}

df_train = df_train.replace({"external_score_ver03": external_score_ver03_dictionary})
df_test = df_test.replace({"external_score_ver03": external_score_ver03_dictionary})

In [8]:
# Concatenate df_train and df_test
df_combined = pd.concat([df_train, df_test], axis=0)

# List of columns to encode
columns_to_encode = ['province', 'juridical_form', 'industry_sector', 'region', 'geo_area', 'decision_date']

# Instantiate LabelEncoder
label_encoders = {}

# Encode each column iteratively
for column in columns_to_encode:
    # Instantiate LabelEncoder for the current column
    label_encoders[column] = LabelEncoder()

    # Fit label encoder on combined data
    label_encoders[column].fit(df_combined[column])

    # Transform both training and testing data
    df_train[column] = label_encoders[column].transform(df_train[column])
    df_test[column] = label_encoders[column].transform(df_test[column])

In [9]:
columns_to_drop = ['application_ID', 'days_to_default']

# Drop the columns from the DataFrame
df_train = df_train.drop(columns=columns_to_drop)
df_train_no_target = df_train.drop(columns=['target'])

df_test = df_test.drop(columns=['application_ID'])

# LGB

In [32]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_train_no_target, df_train.target, test_size=0.2, random_state=42)

# Applying undersampling
undersampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# Creating a LightGBM dataset
train_data = lgb.Dataset(X_resampled, label=y_resampled)

# Setting up parameters for LightGBM
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05,
    'random_state': 42
}

# Training the model
num_round = 3100
bst = lgb.train(params, train_data, num_round)

# Making predictions
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Converting probabilities to binary predictions
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# Calculating F1 score
f1 = f1_score(y_test, y_pred_binary)
print("F1 score:", f1)

[LightGBM] [Info] Number of positive: 20080, number of negative: 20080
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.128861 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6190
[LightGBM] [Info] Number of data points in the train set: 40160, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
F1 score: 0.5977701543739281


# Submit solution

In [26]:
# Making predictions
y_test_pred = bst.predict(df_test, num_iteration=bst.best_iteration)

# Converting probabilities to binary predictions
y_test_pred_binary = [1 if pred > 0.5 else 0 for pred in y_test_pred]

In [27]:
submission = pd.DataFrame(y_test_pred_binary, columns=["label"])

In [28]:
submission.to_csv("file.csv", index=False)